In [2]:
import hashlib
import requests
import datetime
import time
import pandas as pd

In [3]:
def obtener_personaje(): #recuperar los personajes de la API
    url = "https://swapi.dev/api/people/"
    personajes = [] #lista vacía para almacenar todos los personajes 
    while url: #bucle en el que en cada interación el codigo realiza una petición a la API en la URL actual
        response = requests.get(url) #envía la petición GET a la API en la URL y se almacena en response 
        if response.status_code == 200: #200 significa que el resultado get fue exitosa 
            data = response.json() #convierte el contenido en un diccionario de python
            personajes += data['results'] #agrega los personajes a la lista 
            url = data['next']  # pasa a la siguiente pagina, si no hay más, lo detiene 
        else:
            print(f"Error al obtener datos: {response.status_code}") # si la respuesta no tiene un codigo 200, imprime un mensaje de error 
            break
        time.sleep(0.5)  # Pausa para evitar sobrecarga de la API (pausa de 0,5 segundos)
    return personajes

In [4]:

def obtener_homeworld(url):
    try:
        response = requests.get(url) # realiza una solicitud get a la URL y la almacena
        if response.status_code == 200:  #si la función fue exitosa 
            return response.json().get('name', 'Unknown') # obtener el nombre del planeta, sino hay nombre, devolver unknown 
    except Exception as e:
        print(f"Error al obtener homeworld: {e}")
    return 'Unknown'

In [5]:
# Función para obtener la especie
def obtener_especie(urls): #una lista de urls o [](si es humano)
    if not urls:
        return 'Human'  # Suponemos 'Human' si el campo está vacío
    try:
        response = requests.get(urls[0]) #si la lista no esta vacía, acceder a la url 
        if response.status_code == 200:
            return response.json().get('name', 'Unknown') 
    except Exception as e:
        print(f"Error al obtener species: {e}")
    return 'Unknown'

In [6]:
# Función para obtener los títulos de las películas
def obtener_peliculas(urls):
    titulos = [] #lista vacía para almacenar los titulos
    for url in urls: #recorre cada url en la lista
        try:
            response = requests.get(url)
            if response.status_code == 200: #comprueba si la conexión fue exitosa 
                titulos.append(response.json().get('title', 'Unknown')) # si encuentra titulo, sino imprime unknown
        except Exception as e:
            print(f"Error al obtener film: {e}")
    return titulos

In [8]:
def build_dataframe(characters):
    data = [] #lista vacía donde se almacenará los personajes 
    for idx, c in enumerate(characters): #recorre cada personaje (C) de la lista y el idx para el indice 
        
        homeworld = obtener_homeworld(c['homeworld'])
        species = obtener_especie(c['species'])
        films = obtener_peliculas(c['films'])
        
        data.append({
            'id': idx + 1, #para crear un identificador unico 
            'name': c['name'],
            'height': c['height'] or 'Unknown',
            'mass': c['mass'] or 'Unknown',
            'hair_color': c['hair_color'] or 'Unknown',
            'skin_color': c['skin_color'] or 'Unknown',
            'eye_color': c['eye_color'] or 'Unknown',
            'birth_year': c['birth_year'] or 'Unknown',
            'gender': c['gender'] or 'Unknown',
            'homeworld': homeworld,
            'species': species,
            'films': films
        })
        time.sleep(0.5)  
    return pd.DataFrame(data)

# Ejecución del flujo
characters = obtener_personaje()
df = build_dataframe(characters)
df
# Exportar a CSV
#df.to_csv('star_wars_characters.csv', index=False)
#print("Datos exportados a star_wars_characters.csv")

,id,name,height,mass,hair_color,skin_color,eye_color,birth_year,gender,homeworld,species,films
0,1,Luke Skywalker,172,77,blond,fair,blue,19BBY,male,Tatooine,Human,"[A New Hope, The Empire Strikes Back, Return o..."
1,2,C-3PO,167,75,n/a,gold,yellow,112BBY,n/a,Tatooine,Droid,"[A New Hope, The Empire Strikes Back, Return o..."
2,3,R2-D2,96,32,n/a,"white, blue",red,33BBY,n/a,Naboo,Droid,"[A New Hope, The Empire Strikes Back, Return o..."
3,4,Darth Vader,202,136,none,white,yellow,41.9BBY,male,Tatooine,Human,"[A New Hope, The Empire Strikes Back, Return o..."
4,5,Leia Organa,150,49,brown,light,brown,19BBY,female,Alderaan,Human,"[A New Hope, The Empire Strikes Back, Return o..."
...,...,...,...,...,...,...,...,...,...,...,...,...
77,78,Grievous,216,159,none,"brown, white","green, yellow",unknown,male,Kalee,Kaleesh,[Revenge of the Sith]
78,79,Tarfful,234,136,brown,brown,blue,unknown,male,Kashyyyk,Wookie,[Revenge of the Sith]
79,80,Raymus Antilles,188,79,brown,light,brown,unknown,male,Alderaan,Human,"[A New Hope, Revenge of the Sith]"
80,81,Sly Moore,178,48,none,pale,white,unknown,female,Umbara,Human,"[Attack of the Clones, Revenge of the Sith]"
